In [15]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
import pandas as pd

In [16]:
def crawling(url, xlsxName):
    ## Opening chrome and url

    option = webdriver.ChromeOptions()
    option.add_argument('--headless')
    option.add_argument('-no-sandbox')
      # option.add_argument("--disable-infobars")
      # option.add_argument("--disable-gpu")
    option.add_argument("--mute-audio")
    option.add_argument("--disable-extensions")
    option.add_argument('-disable-dev-shm-usage')

    driver = webdriver.Chrome(options=option) # 배포용 (브라우저 미출력)
    #driver = webdriver.Chrome() # 테스트용 (브라우저 출력)


    driver.set_window_size(960, 800)  # 유튜브 디자인이 해상도 크기에 따라 달라지므로 최소값으로 설정하고 진행
    # 최소값으로 설정하지 않으면 추천 동영상 부분이 오른쪽에 나타나는데, 그러면 스크롤 처리 부분에서 부정확할 수 있음


    time.sleep(1)
    driver.get(url)
    time.sleep(2)

    ## 유튜브 영상 일시정지 진행

    pause = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'ytp-play-button')))

    pause.click()
    time.sleep(0.2)
    pause.click()
    time.sleep(4)

    ## 댓글 수집을 위한 스크롤 이벤트 발생

    # 댓글을 나타나게 하기 위핸 첫 스크롤
    driver.execute_script("window.scrollBy(0,500)", "")

    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    # 테스트용
    count = 0

    while True:
        # 현재 페이지 길이 만큼 스크롤 진행
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        # 댓글 로딩 시간 기다림
        time.sleep(4)

        # 스크롤 진행 후 새로운 페이지 길이를 측정
        new_height = driver.execute_script("return document.documentElement.scrollHeight")

        # 만약 새로운 페이지 길이가 기존과 동일하다면, 스크롤이 되지 않았다는 의미 = 모든 댓글을 불러왔다는 의미
        if new_height == last_height:
            break
        last_height = new_height

        # 테스트용
        # count += 1
        # if count >3:
        #     break

    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

    ## 댓글 정보 가져오기


    ## 영상 업로드 시간
    video_time = driver.find_element(By.XPATH, '//ytd-watch-info-text/tp-yt-paper-tooltip/div')
    video_time = video_time.get_attribute("innerHTML").split('•')[1].strip()


    ## 영상 제목
    video_title = driver.find_element(By.NAME, 'title').get_attribute('content')


    ## 영상 소유자
    video_owner = driver.find_element(By.XPATH, '//*[@id="text"]/a')


    ## 댓글 정보
    users = driver.find_elements(By.XPATH, '//*[@id="author-text"]/span')
    comments = driver.find_elements(By.XPATH, '//*[@id="content-text"]')
    likes = driver.find_elements(By.XPATH, '//*[@id="like-button"]/yt-button-shape/button/@aria-label')
    times = driver.find_elements(By.XPATH, '//*[@id="header-author"]/yt-formatted-string/a')

    # 확인용으로 배열의 길이를 출력
    print("Users count:", len(users))
    print("Comments count:", len(comments))
    print("Likes count:", len(likes))
    print("Times count:", len(times))


    data = []


    ## 해당 정보를 묶어서 xlsx파일로 생성
    for username, comment, like, time_ in zip(users, comments, likes, times):
        data.append({
            'owner': video_owner.text,
            'title': video_title,
            'Username': username.text,
            'Comment': comment.text,
            'Likes': like.text,
            'Times': video_time
        })

    df = pd.DataFrame(data)
    
    print(df)

    df.to_excel(xlsxName, engine='openpyxl')

    ## 드라이버 종료
    driver.close()

    return

In [17]:
link_dict = {
    "국가애도기간": [
        "https://www.youtube.com/watch?v=cnNzZcjuLlk",
        "https://www.youtube.com/watch?v=LWVR_OenAYo",
        "https://www.youtube.com/watch?v=WzxQgHD8DSw",
        "https://www.youtube.com/watch?v=8IHW57wM6s4",
        "https://www.youtube.com/watch?v=qD2mI0jlxZ8"
    ],

    "용어지침": [
        "https://www.youtube.com/watch?v=MH5WgOLINx4",
        "https://www.youtube.com/watch?v=xtbsWabMAbY",
        "https://www.youtube.com/watch?v=lE5aBIt_iyE",
        "https://www.youtube.com/watch?v=cGQ_ijmXzFA"
    ],

    "대통령사과": [
        "https://www.youtube.com/watch?v=6mHy4QrY7qU",
        "https://www.youtube.com/watch?v=vBESJUNPI4U",
        "https://www.youtube.com/watch?v=7ZU_Lw0FSvA",
        "https://www.youtube.com/watch?v=FkIlWc-31mM",
        "https://www.youtube.com/watch?v=IQdtAZ6N8YM"
    ],

    "경찰책임": [
        "https://www.youtube.com/watch?v=8IessBIIqEQ",
        "https://www.youtube.com/watch?v=E03ZtnYJXeo",
        "https://www.youtube.com/watch?v=CvYfo25oZ9c",
        "https://www.youtube.com/watch?v=zLK2UuFDiCI",
        "https://www.youtube.com/watch?v=tlPGiUokn9k",
        "https://www.youtube.com/watch?v=2UO6Ar6dluE",
        "https://www.youtube.com/watch?v=Tcp9i1gFeJU"
    ],

    "정부책임": [
        "https://www.youtube.com/watch?v=tfTX5W-i9lw",
        "https://www.youtube.com/watch?v=qhtk4-riVfc",
        "https://www.youtube.com/watch?v=P-1Yck7TqKo",
        "https://www.youtube.com/watch?v=GwZ7-POnVEA"
    ],

    "유가족협의회": [
        "https://www.youtube.com/watch?v=jeVdUO5iDsU",
        "https://www.youtube.com/watch?v=sub9Lrz_lf0",
        "https://www.youtube.com/watch?v=OUQ0Y3LUU70",
        "https://www.youtube.com/watch?v=oH1-yXvPLs0",
        "https://www.youtube.com/watch?v=cuHpCkzkIoY"
    ],

    "특별법": [
        "https://www.youtube.com/watch?v=bOcpfYYmYvg",
        "https://www.youtube.com/watch?v=QlzsnvPYdtU",
        "https://www.youtube.com/watch?v=6tNm5c-JPko",
        "https://www.youtube.com/watch?v=_qIuNCHZCPU",
        "https://www.youtube.com/watch?v=qSFacPC-03s"
    ]
}

In [18]:
for key in link_dict.keys():
    i = 0
    for link in link_dict[key]:
        i += 1
        crawling(link, f"1_crawled/{key}{i}.xlsx")
        print(f"1_crawled/{key}{i}.xlsx파일 크롤링 완료")

Users count: 0
Comments count: 102
Likes count: 0
Times count: 102
Empty DataFrame
Columns: []
Index: []
1_crawled/국가애도기간1.xlsx파일 크롤링 완료
Users count: 0
Comments count: 31
Likes count: 0
Times count: 31
Empty DataFrame
Columns: []
Index: []
1_crawled/국가애도기간2.xlsx파일 크롤링 완료


KeyboardInterrupt: 